# Finetuning Bart large on CNN daily news dataset

In [ ]:
### MODULES ###

import sys,os
import tqdm
import csv
import numpy as np
import pandas as pd
import json
import torch

from datasets import load_dataset

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

# Load the ROUGE metric
import evaluate

from transformers import AutoTokenizer, BartForConditionalGeneration

# Load the ROUGE metric
import evaluate


#link : https://github.com/facebookresearch/fairseq/tree/main/examples/bart
#link : https://huggingface.co/docs/transformers/model_doc/bart
#link : https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/model#transformers.PreTrainedModel

In [16]:

NUM_PROCS = os.cpu_count() 

print("NUM_PROCS = " ,NUM_PROCS)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


NUM_PROCS =  12
cuda


In [17]:
# Load configuration from JSON
with open('./config_finetune_bart_large.json', 'r') as f:
    config = json.load(f)
    print(config)
    print(type(config))

SEED = config['config_machine']["SEED"]
NUM_LOADER = 4 #config['config_machine']["NUM_LOADER"] #depends of the number of thread 


# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(SEED) # pytorch random seed
np.random.seed(SEED) # numpy random seed
torch.backends.cudnn.deterministic = True

{'config_machine': {'SEED': 42, 'NUM_LOADER': 50}, 'config_model': {'MODEL_TYPE': 'bart', 'MODEL_NAME': 'bart-large', 'MODEL_HUB': 'facebook/bart-large'}, 'config_generate': {'NUM_BEAM': 4, 'min_len_resume': 14, 'max_len_resume': 159, 'no_repeat_ngram_size': 3, 'repetition_penalty': 1.5, 'length_penalty': 2, 'early_stopping': True, 'use_cache': False}, 'config_training': {'max_len': 1024, 'TRAIN_BATCH_SIZE': 16, 'VALID_BATCH_SIZE': 32, 'LEARNING_RATE': 0.0001, 'NB_EPOCHS': 15, 'early_stopping_patience': 5, 'reduce_lr_patience': 2, 'reduce_lr_factor': 0.1}}
<class 'dict'>


# Load dataset CNN daily

In [4]:
# Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")

# Check the dataset structure
print(dataset)

## Comment this part for the real training time :

percentage = 0.05

for split in dataset: 
    dataset[split] = dataset[split].shuffle(seed=SEED).select(range(int(len(dataset[split]) * percentage)))

# Check the dataset structure
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})
DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 14355
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 668
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 574
    })
})


# load the model and tokenizer 

In [20]:
### Load model ###
from transformers import BartForConditionalGeneration, AutoTokenizer

MODEL_HUB = config["config_model"]["MODEL_HUB"]
# Load Model and Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_HUB, clean_up_tokenization_spaces=True)
model = BartForConditionalGeneration.from_pretrained(MODEL_HUB, forced_bos_token_id=0)
print(tokenizer.model_max_length)

1024


In [11]:
def len_distrib(batch):

    len_articles = []
    len_highlights = []
    
    for article, highlight in zip(batch["article"], batch["highlights"]):
        len_articles.append(len(tokenizer(article, truncation=False)["input_ids"]))
        len_highlights.append(len(tokenizer(highlight, truncation=False)["input_ids"]))


    source = tokenizer(batch["article"],truncation=True, max_length=tokenizer.model_max_length)
    resume = tokenizer(batch["highlights"],truncation=True, max_length=tokenizer.model_max_length)

    return {
        'input_ids': source['input_ids'], 
        'input_mask': source['attention_mask'],
        'input_len': len_articles,
        'target_ids': resume['input_ids'], 
        'target_mask': resume['attention_mask'],
        'target_len': len_highlights
        }


dataset = dataset.map(len_distrib,num_proc=NUM_PROCS,batched=True,batch_size=64)# Save the Hugging Face dataset


Map (num_proc=12):   0%|          | 0/14355 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1153 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1142 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1105 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1498 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2187 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

Map (num_proc=12):   0%|          | 0/668 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1380 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1752 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1064 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1048 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2112 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

Map (num_proc=12):   0%|          | 0/574 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1647 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1025 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1592 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1917 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1153 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence leng

In [ ]:
## TO CONTINUE

# Define the custom collate function
def collate_fn(batch):
    """
    Custom collate function that add padding for each batch.
    """

    id = [item['id'] for item in batch]

    # Pad the tokenized content
    padded_text_ids = pad_sequence(
        [torch.tensor(item['input_ids'], dtype=torch.long) for item in batch], 
        batch_first=True, 
        padding_value=tokenizer.pad_token_id)
    
    padded_text_mask = pad_sequence(
        [torch.tensor(item['input_mask'], dtype=torch.long) for item in batch], 
        batch_first=True, 
        padding_value=0)

    decoder_input_ids = pad_sequence(
        [torch.tensor(item['target_ids'], dtype=torch.long) for item in batch], 
        batch_first=True, 
        padding_value=tokenizer.pad_token_id)     
    
    decoder_attention_mask = pad_sequence(
        [torch.tensor(item['target_mask'], 
                      dtype=torch.long) for item in batch], 
                      batch_first=True, 
                      padding_value=0)
    
    input_len = [item['input_len'] for item in batch]

    target_len = [item['target_len'] for item in batch]

    highlights = [item['highlights'] for item in batch] 
    

    return {
        'id':id,
        'input_ids':padded_text_ids,
        'attention_mask':padded_text_mask,
        'decoder_input_ids':decoder_input_ids,
        'target_mask':decoder_attention_mask,
        'input_len': input_len ,
        'target_len': target_len,
        'highlights': highlights
    }


train_params = {
    'batch_size': config["config_training"]["TRAIN_BATCH_SIZE"],
    'shuffle': False,
    'collate_fn':collate_fn,
    'num_workers': NUM_LOADER,
    'pin_memory': True  #  Enables faster GPU transfers
    }

# This will be used down for training and validation stage for the model.
train_loader = DataLoader(dataset["train"], **train_params)

for batch in train_loader:
    print(batch)
    break


{'id': ['08cf276c9eadb638e0c7fdc83ce0229c8af5d09b', 'a0965f34cb08bd7db5845f8285dc8a9512d3e590', 'e960738caa9d39737f459e39923b20160f73f258', '9befc7ac904202d6e018093c2923f04479d6af05', 'dc3c9f319ca17b33ad87c71049a8589e474ba769', 'a6c74feb871eb462a645b442d23afb0098fd304e', '6182e63f54f7d4fc1e03caeb752d69e784c531a4', 'b2d77b13bd8aaf7ae9ba8f1e2771e115dd665698', '4bb5657c537eb9f8d00876985b91c578ab1ec3d8', '40d500f482a41b209ff379f2207180b71dd3bf35', '557fd3f6fc104f27dc6071bd80dc7c0b2c19cafa', '0bb9c54124e50dabcdc9c44828470b30024657b0', '4a0caecbc2bdd104e56828bc68a8244f1b3743f6', '427816b50f73c14cd2b6ea9dc088ddeb4e7f72a3', 'bc98b9b915afcc5a9fe1d1fb868ee9b546cfda06', 'ccc3717dca507094ffba8a59d8185bd6b0ad8d4a'], 'input_ids': tensor([[    0,  2765,   479,  ...,     1,     1,     1],
        [    0,  4154, 33957,  ..., 13223,  2637,     2],
        [    0, 45158,    12,  ...,     1,     1,     1],
        ...,
        [    0,  1640, 16256,  ...,     1,     1,     1],
        [    0, 24485,  1422,